In [15]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [35]:
df = pd.read_csv('data/stock_returns.csv')[['tic', 'date', 'close']].rename(columns={'tic': 'Ticker',
                                                                                     'close': 'Price',
                                                                                     'date': 'Date'})

In [36]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df.sort_values(by=['Ticker', 'Date'], inplace=True)

In [37]:
df['Returns'] = df.groupby('Ticker')['Price'].pct_change()

In [38]:
window= 252

#### Utils

In [39]:
def portfolio_variance(weights, cov_matrix):
    return weights.dot(cov_matrix).dot(weights.T)

In [40]:
def mean_variance_optimization(returns_df, initial_weights):

    # Calculate the covariance matrix
    cov_matrix = returns_df.cov()   

    # Constraints: weights sum to 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) 

    # Bounds for weights: can't hold more than 100% of any stock
    bounds = tuple((0, 1) for _ in range(len(initial_weights)))

    # Perform optimization to minimize portfolio variance
    result = minimize(portfolio_variance, initial_weights, args=(cov_matrix,),
                      method='SLSQP', bounds=bounds, constraints=constraints)

    return result.x if result.success else None

#### State Variables

In [41]:
# Initialize a dictionary to hold the allocations
allocations = {}

# Get unique dates
unique_dates = df['Date'].unique()

# Get unique tickers
tickers = df['Ticker'].unique()

# Initial weights (equal distribution)
initial_weights = np.array([1/len(tickers)] * len(tickers))

In [42]:
df = df[df['Date'] > pd.Timestamp('2019-12-31')]

#### Mean Variance Optimization

In [43]:
for date in unique_dates:

    # Filter the DataFrame for the current date and past year
    start_date = date - pd.DateOffset(years=1)
    past_year_data = df[(df['Date'] > start_date) & (df['Date'] <= date)]
   
    # Skip dates with insufficient data
    if past_year_data['Date'].nunique() < window:
        continue

    # Calculate the moving average of returns for the past year
    moving_avg_returns = past_year_data.groupby('Ticker')['Returns'].mean()

    # Create a DataFrame for the returns
    returns_df = past_year_data.pivot(index='Date', columns='Ticker', values='Returns').dropna()

    # Perform MVO
    optimal_weights = mean_variance_optimization(returns_df, initial_weights)

    # Store the allocations
    allocations[date] = dict(zip(tickers, optimal_weights))

In [45]:
# Convert allocations to a DataFrame
allocations_df = pd.DataFrame.from_dict(allocations, orient='index')

In [46]:
print(allocations_df)

                AAPL      AMGN           AXP            BA       CAT  \
2020-12-30  0.004848  0.072195  3.740497e-18  0.000000e+00  0.003625   
2020-12-31  0.005087  0.072098  0.000000e+00  1.409463e-18  0.003526   
2021-01-04  0.005037  0.071643  4.662069e-18  1.317306e-17  0.004203   
2021-01-05  0.005027  0.071620  0.000000e+00  0.000000e+00  0.004211   
2021-01-06  0.004957  0.071160  3.794708e-18  1.894643e-17  0.001927   
...              ...       ...           ...           ...       ...   
2022-12-23  0.033333  0.033333  3.333333e-02  3.333333e-02  0.033333   
2022-12-27  0.033333  0.033333  3.333333e-02  3.333333e-02  0.033333   
2022-12-28  0.033333  0.033333  3.333333e-02  3.333333e-02  0.033333   
2022-12-29  0.033333  0.033333  3.333333e-02  3.333333e-02  0.033333   
2022-12-30  0.033333  0.033333  3.333333e-02  3.333333e-02  0.033333   

                 CRM      CSCO           CVX       DIS           DOW  ...  \
2020-12-30  0.025885  0.014748  1.951564e-18  0.000599  2.

#### Visualizations